In [1]:
from imutils import paths
import face_recognition
import pickle
import cv2
import os
import time

In [5]:
%pwd

'/home/ebi/machinelearning/end_to_end_ml_projects/ete_face_recognition/research'

In [6]:
%cd ../

/home/ebi/machinelearning/end_to_end_ml_projects/ete_face_recognition


/home/ebi/machinelearning/end_to_end_ml_projects/ete_face_recognition/venv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [7]:
%pwd

'/home/ebi/machinelearning/end_to_end_ml_projects/ete_face_recognition'

In [23]:
args = {}
args['dataset'] = os.path.join("artifacts", "data_ingestion", "dataset")
args['encoding'] = os.path.join("artifacts", "trained_model", "encoding.pickle")
args['detection_method'] = 'cnn'

os.getcwd()

'/home/ebi/machinelearning/end_to_end_ml_projects/ete_face_recognition'

## Create facial embedding

In [24]:
def create_facial_embeddings(args):
    # grab the path to the input images in our dataset
    print("[INFO] quantifing faces...")
    imagePaths = list(paths.list_images(args['dataset']))
    # initialize the list of known encoding and known names
    KnownEncodings = []
    KnownNames = []
    for (i, imagePath) in enumerate(imagePaths):
        print(i, imagePath)
    # dlib and gace_recognition are in RGB color channel order
    print('[INFO] processing image...')
    ti = time.time()
    # loop over the image paths
    for (i, imagePath) in enumerate(imagePaths):
        #extract the person name from the image path
        print('{}/{}'.format(i+1, len(imagePaths)), end=', ')
        name = imagePath.split(os.path.sep)[-2]
        # load the input image and convert it to RGB
        image = cv2.imread(imagePath)
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #detect the (x, y)-coordinates of the bounding boxes corresponding
        # to each face in the image
        boxes = face_recognition.face_locations(rgb, model=args['detection_method'])
        #compute the facial embedding for the face, ie, to turn the
        # bounding boxes of the face into a list of 128 numbers
        encodings = face_recognition.face_encodings(rgb, boxes)
        #loop over the encodings
        for encoding in encodings:
            #add each encoding + name to our set of known names and encodings
            KnownEncodings.append(encoding)
            KnownNames.append(name)
    print('Done!')
    print('Time taken: {:.1f} minutes'.format((time.time() - ti)/60))
    #dump the names and encodings to disk for future recall
    #encoding.pickle contains the 128-d face embeddings for each 
    # face in our dataset
    print('[INFO] serializing encodings...')
    data = {'encoding': KnownEncodings, 'names':KnownNames}
    f = open(args['encoding'], 'wb')
    f.write(pickle.dumps(data))
    f.close()
    print('Done!') 

In [25]:
create_facial_embeddings(args)

[INFO] quantifing faces...
0 artifacts/data_ingestion/dataset/john_hammond/John Hammond (2).jpg
1 artifacts/data_ingestion/dataset/john_hammond/John Hammond (3).jpg
2 artifacts/data_ingestion/dataset/john_hammond/John Hammond (5).jpg
3 artifacts/data_ingestion/dataset/john_hammond/John Hammond (4).jpg
4 artifacts/data_ingestion/dataset/john_hammond/John Hammond (1).jpg
5 artifacts/data_ingestion/dataset/owen_grady/Owen Grady (3).jpg
6 artifacts/data_ingestion/dataset/owen_grady/Owen Grady (1).jpg
7 artifacts/data_ingestion/dataset/owen_grady/Owen Grady (5).jpg
8 artifacts/data_ingestion/dataset/owen_grady/Owen Grady (4).jpg
9 artifacts/data_ingestion/dataset/owen_grady/Owen Grady (2).jpg
10 artifacts/data_ingestion/dataset/ian_malcolm/Ian_Malcolm_(Jeff_Goldblum).jpg
11 artifacts/data_ingestion/dataset/ian_malcolm/Ian Malcolm (2).jpg
12 artifacts/data_ingestion/dataset/ian_malcolm/Ian Malcolm (4).jpg
13 artifacts/data_ingestion/dataset/ian_malcolm/Ian Malcolm (1).jpg
14 artifacts/data_i

## Recognize faces in images

In [37]:
import face_recognition
import pickle
import cv2
import os
import time
from collections import Counter

args['image'] = os.path.join("artifacts","data_ingestion","dataset",   'alan_grant', "Alan Grant (2).jpg")
print(args['image'])
args['detection_method2'] = 'hog'
args['output'] = os.path.join("artifacts", "outputs")

artifacts/data_ingestion/dataset/alan_grant/Alan Grant (2).jpg


In [34]:
def recognise_faces(args):
    ti = time.time()
    #load the known faces and embeddings
    print('[INFO] loading encodings...')
    data = pickle.loads(open(args['encoding'], 'rb').read())
    #load the input image and convert it from BGR to RGB
    image = cv2.imread(args['image'])
    print(image.shape)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #detect the (x, y)-coordinates of the bounding boxes 
    print('[INFO] recognising faces...')
    boxes = face_recognition.face_locations(rgb, model=args['detection_method2'])
    encodings = face_recognition.face_encodings(rgb, boxes)
    #initialize the list of names for each face detected
    names = []
    
    #loop over the facial embeddings
    for encoding in encodings:
        #attempt to match each face in the input image to
        # our known encodings, function returns a list of
        # True/False values, one for each known encoding
        #Internally, the compare_faces function is computing
        # the Euclidean distance between the candidate embedding
        # and all faces in our known encodings
        votes = face_recognition.compare_faces(data['encoding'], encoding)
        #check to see if a match is found
        if True in votes:
            #find the corresponding names of all faces matched(vote = True)
            matches = [name for name, vote in list(zip(data['names'], votes)) if vote == True]
            #determine the most frequently occuring name
            name = Counter(matches).most_common()[0][0]
        else:
            name = "Unknown"
        #update the list of names
        names.append(name)
    print([' '.join([e.title() for e in name.split('_')]) for name in names])
    print('Time taken: {:.1f} seconds'.format(time.time() - ti))
    
    #visualise with bounding boxes and labeled names, loop over the recognised faces
    for ((top, right, bottom, left), name) in zip(boxes, names):
        #draw the predicted face name on the image
        cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
        y = top - 15 if top - 15 > 15 else top + 15
        cv2.putText(image, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
        
    #display the resulting frame, press 'q' to exit
    window_text = args['image'].split(os.path.sep)[-1]
    cv2.imshow(window_text, image)
    while True:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break
    #save output image
    cv2.imwrite(args['output'] + '_output.jpg', image)

In [38]:
recognise_faces(args)

[INFO] loading encodings...
(464, 610, 3)
[INFO] recognising faces...
['Alan Grant']
Time taken: 0.2 seconds
